In [74]:
import pandas as pd
import sqlite3

In [75]:
df = pd.read_csv('../german_credit_augmented.csv')

In [76]:
con = sqlite3.connect('db')

In [77]:
df.dtypes

age                  int64
sex                 object
job                  int64
housing             object
saving_accounts     object
checking_account    object
credit_amount        int64
duration             int64
purpose             object
default              int64
contract_dt         object
client_id            int64
dtype: object

In [78]:
df.head()

,age,sex,job,housing,saving_accounts,checking_account,credit_amount,duration,purpose,default,contract_dt,client_id
0,33,male,2,own,NaN,NaN,3074,9,radio/TV,0,2008-06-29 18:52:00,210
1,43,male,1,own,little,little,1344,12,car,0,2007-05-20 18:30:19,929
2,52,male,2,own,quite rich,NaN,936,9,education,0,2008-04-27 08:23:07,200
3,35,female,3,own,little,NaN,1393,11,car,0,2007-05-06 10:58:22,45
4,28,male,2,own,little,NaN,776,12,radio/TV,0,2007-07-21 13:22:14,358


In [79]:
df['contract_dt'] = pd.to_datetime(df['contract_dt'], format="%Y-%m-%d %H:%M:%S")

In [80]:
df.dtypes

age                          int64
sex                         object
job                          int64
housing                     object
saving_accounts             object
checking_account            object
credit_amount                int64
duration                     int64
purpose                     object
default                      int64
contract_dt         datetime64[ns]
client_id                    int64
dtype: object

In [81]:
df

,age,sex,job,housing,saving_accounts,checking_account,credit_amount,duration,purpose,default,contract_dt,client_id
0,33,male,2,own,NaN,NaN,3074,9,radio/TV,0,2008-06-29 18:52:00,210
1,43,male,1,own,little,little,1344,12,car,0,2007-05-20 18:30:19,929
2,52,male,2,own,quite rich,NaN,936,9,education,0,2008-04-27 08:23:07,200
3,35,female,3,own,little,NaN,1393,11,car,0,2007-05-06 10:58:22,45
4,28,male,2,own,little,NaN,776,12,radio/TV,0,2007-07-21 13:22:14,358
...,...,...,...,...,...,...,...,...,...,...,...,...
995,65,male,2,free,little,little,2600,18,radio/TV,1,2007-12-16 20:17:19,624
996,30,male,3,own,little,moderate,4455,36,business,1,2007-07-12 14:08:58,181
997,33,male,2,own,little,moderate,6403,24,radio/TV,0,2008-04-08 03:24:26,730
998,29,female,2,own,NaN,NaN,5003,21,car,1,2007-11-29 15:51:45,557


In [82]:
df.to_sql('german_credit', con, index=False, if_exists='replace')

DatabaseError: Execution failed on sql 'DROP TABLE "german_credit"': database is locked

In [ ]:
def select(sql):
    return pd.read_sql(sql, con)

In [ ]:
# sql = '''select *, t.age*3 as age_mult3, age*1.0/credit_amount as age_div_credit from german_credit as t'''

In [ ]:
sql = '''select * from german_credit as t
        where t.age between 30 and 40
        and t.purpose in ('car','business')
        order by t.age desc, t.credit_amount
        '''

In [ ]:
select(sql)

,age,sex,job,housing,saving_accounts,checking_account,credit_amount,duration,purpose,default,contract_dt,client_id
0,40,female,2,own,rich,moderate,1322,11,car,0,2007-11-14 02:55:59,875
1,40,male,1,own,little,little,1361,6,car,0,2008-02-25 02:24:28,516
2,40,male,1,rent,quite rich,None,1597,10,car,0,2008-06-08 12:22:38,413
3,40,male,1,own,None,little,1647,21,car,1,2007-08-16 09:28:14,182
4,40,male,0,own,little,moderate,3590,18,business,0,2008-02-01 17:14:11,950
...,...,...,...,...,...,...,...,...,...,...,...,...
162,30,male,2,own,little,moderate,7966,26,car,0,2007-11-08 13:56:18,525
163,30,female,2,own,little,None,8133,36,car,0,2008-06-03 19:03:00,70
164,30,female,2,own,quite rich,moderate,8358,48,car,0,2008-04-30 16:52:53,945
165,30,male,3,own,None,None,11054,36,car,0,2008-05-04 05:03:38,450


In [ ]:
car_data = pd.DataFrame({'purpose':['машина', 'машины', 'у машин', 'автомобиль', 'для салона'],
             'amount':[1000, 200, 3000, 2300, 100]})

In [ ]:
car_data.to_sql('car_purpose', con, index = False, if_exists='replace')

DatabaseError: Execution failed on sql 'DROP TABLE "car_purpose"': database is locked

In [ ]:
sql = '''select * from car_purpose'''

In [ ]:
select(sql)

,purpose,amount
0,машина,1000
1,машины,200
2,у машин,3000
3,автомобиль,2300
4,для салона,100


In [ ]:
sql = '''select * from car_purpose as t
        where (t.purpose like '%машин%' or t.purpose like '%авто%') and amount > 500
'''

In [ ]:
select(sql)

,purpose,amount
0,машина,1000
1,у машин,3000
2,автомобиль,2300


In [ ]:
# получит средные значения 

In [ ]:
sql = '''select avg(case when t.credit_amount > 1000 then 1 else 0 end) as avg_greater_then_1000 from german_credit t
    '''

In [ ]:
select(sql)

,avg_greater_then_1000
0,0.884


In [ ]:
sql = '''select t.credit_amount, 
    iif(t.credit_amount > 1000, 1, 0) as greater_then_1000
from german_credit t '''

In [ ]:
select(sql)

,credit_amount,greater_then_1000
0,3074,1
1,1344,1
2,936,0
3,1393,1
4,776,0
...,...,...
995,2600,1
996,4455,1
997,6403,1
998,5003,1


In [ ]:
# create new table in sqllite3

sql = '''
    drop table if exists greater_1000_credit;
    create table greater_1000_credit as
    select * from german_credit t
    where t.credit_amount > 1000
    '''

In [ ]:
# select(sql)

In [83]:
cur = con.cursor()

In [84]:
cur.executescript(sql)

In [85]:
sql = '''select * from greater_1000_credit t'''

In [86]:
select(sql)

,age,sex,job,housing,saving_accounts,checking_account,credit_amount,duration,purpose,default,contract_dt,client_id
0,33,male,2,own,None,None,3074,9,radio/TV,0,2008-06-29 18:52:00,210
1,43,male,1,own,little,little,1344,12,car,0,2007-05-20 18:30:19,929
2,35,female,3,own,little,None,1393,11,car,0,2007-05-06 10:58:22,45
3,27,female,2,own,little,moderate,1295,18,furniture/equipment,0,2008-06-18 04:10:05,86
4,26,male,2,own,little,little,4370,42,radio/TV,1,2007-11-29 00:20:44,639
...,...,...,...,...,...,...,...,...,...,...,...,...
879,65,male,2,free,little,little,2600,18,radio/TV,1,2007-12-16 20:17:19,624
880,30,male,3,own,little,moderate,4455,36,business,1,2007-07-12 14:08:58,181
881,33,male,2,own,little,moderate,6403,24,radio/TV,0,2008-04-08 03:24:26,730
882,29,female,2,own,None,None,5003,21,car,1,2007-11-29 15:51:45,557
